# Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import librosa as lr

In [2]:
data_folder = "/Users/harsh/Downloads/Audio_1/LibriSpeech/"

In [3]:
x=np.array(())
y=np.array(())
file = open(data_folder+'SPEAKERS.txt','r')
for i in range(12):
    file.readline()
line = file.readline().split('|')
Dict = {'F':0,'M':1}
while (len(line)>1):
    package = line[2].strip()
    if(package=='dev-clean'):
        y = np.append(y,Dict.get(line[1].strip()))
        x = np.append(x,int(line[0].strip()))
    line = file.readline().split('|')
file.close() 
data = {}
for i in range(len(x)):
    data[int(x[i])] = np.array(())

In [4]:
#y,x

In [5]:
#data = np.array(())

In [6]:
#from progress.bar import Bar

In [7]:
j=0
for subdirs,dirs,files in os.walk(data_folder+'dev-clean'):
    for file in files:
        if file.endswith('.flac'):
            audio,sf = lr.load(subdirs+'/'+file)
            data[int(subdirs.split('/')[-2])] = np.append(data[int(subdirs.split('/')[-2])],audio)    
    j+=1
    print(float(j)/181*100)
            


0.5524861878453038
1.1049723756906076
1.6574585635359116
2.209944751381215
2.7624309392265194
3.314917127071823
3.867403314917127
4.41988950276243
4.972375690607735
5.524861878453039
6.077348066298343
6.629834254143646
7.18232044198895
7.734806629834254
8.287292817679557
8.83977900552486
9.392265193370166
9.94475138121547
10.497237569060774
11.049723756906078
11.602209944751381
12.154696132596685
12.70718232044199
13.259668508287293
13.812154696132598
14.3646408839779
14.917127071823206
15.469613259668508
16.022099447513813
16.574585635359114
17.12707182320442
17.67955801104972
18.23204419889503
18.784530386740332
19.337016574585636
19.88950276243094
20.441988950276244
20.994475138121548
21.54696132596685
22.099447513812155
22.65193370165746
23.204419889502763
23.756906077348066
24.30939226519337
24.861878453038674
25.41436464088398
25.96685082872928
26.519337016574585
27.071823204419886
27.624309392265197
28.176795580110497
28.7292817679558
29.2817679558011
29.83425414364641
30.386740

In [8]:
#plt.plot(np.arange(0,len(data[x[1]]))/sf,data[x[1]])

In [9]:
bins = 512
data_seg = {}
seg = 43*5
fft = np.zeros((seg,bins//2),dtype='complex')
for i in range(len(x)):
    lfft = len(data[x[i]])//bins 
    data_seg[int(x[i])] = lfft//seg
    l = 0
    for k in range(lfft//seg):
        while(l<seg*(k+1)):
            fft[l%seg] = np.abs(np.fft.fftshift(np.fft.fft(data[x[i]][l*bins:(l+1)*bins]))[bins//2:])*2/bins
            l+=1
        fig = plt.figure(k,frameon=False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(np.abs(fft.T),aspect='auto',clim=[0,5e-3],extent=[1,seg,bins//2,1])
        ax.set_yscale('log')
        if(y[i]==0):
            fig.savefig(data_folder+'dev-clean/Spectrograms/Female/'+str(int(x[i]))+'_png/'+str(k)+'.png')
        else:
            fig.savefig(data_folder+'dev-clean/Spectrograms/Male/'+str(int(x[i]))+'_png/'+str(k)+'.png')
        plt.close()
    print(float(i+1)/len(x)*100)

2.5
5.0
7.5
10.0
12.5
15.0
17.5
20.0
22.5
25.0
27.500000000000004
30.0
32.5
35.0
37.5
40.0
42.5
45.0
47.5
50.0
52.5
55.00000000000001
57.49999999999999
60.0
62.5
65.0
67.5
70.0
72.5
75.0
77.5
80.0
82.5
85.0
87.5
90.0
92.5
95.0
97.5
100.0


In [10]:
#import cv2

In [11]:
#for subdirs,dirs,files in os.walk(data_folder+'dev-clean'):
#    for file in files:
#        if file.endswith('.png'):
#            image = cv2.imread(subdirs+'/'+file)
            
            #data[int(subdirs.split('/')[-2])] = np.append(data[int(subdirs.split('/')[-2])],audio)    

# CNN

In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [33]:
train_gen = ImageDataGenerator(rescale=1./255,validation_split=0.1)
train_set = train_gen.flow_from_directory(data_folder+'dev-clean/Spectrograms/',subset='training',target_size=(512,512),color_mode='rgb',class_mode='binary',batch_size=50)
test_set = train_gen.flow_from_directory(data_folder+'dev-clean/Spectrograms/',subset='validation',target_size=(512,512),color_mode='rgb',class_mode='binary',batch_size=50)

Found 11655 images belonging to 2 classes.
Found 1294 images belonging to 2 classes.


In [42]:
inpt = Input((512,512,3))

In [43]:
c1 = Conv2D(8,(3,3),activation='relu',padding='same')(inpt)
#c12 = Conv2D(8,(3,3),activation='relu',padding='same')(c1)
m1 = MaxPooling2D((2,2))(c1)
c2 = Conv2D(16,(3,3),activation='relu',padding='same')(m1)
#c22 = Conv2D(16,(3,3),activation='relu',padding='same')(c2)
m2 = MaxPooling2D((2,2))(c2)
c3 = Conv2D(32,(3,3),activation='relu',padding='same')(m2)
#c32 = Conv2D(32,(3,3),activation='relu',padding='same')(c3)
m3 = MaxPooling2D((2,2))(c3)

In [44]:
f = Flatten()(m3)

In [45]:
d1 = Dense(output_dim=64,activation='relu')(f)
#drp1 = Dropout(0.2)(d1)
#d2 = Dense(output_dim=64,activation='relu')(d1)
#drp2 = Dropout(0.5)(d2)
outpt = Dense(output_dim=1,activation='sigmoid')(d1)

/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  """


In [46]:
model = Model(inpt,outpt)

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 512, 512, 8)       224       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 256, 256, 8)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 256, 256, 16)      1168      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 64, 64, 32)        0         
__________

In [50]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [51]:
model.fit_generator(train_set,steps_per_epoch=220,epochs=3,validation_data=test_set,validation_steps=22)

Epoch 1/3
220/220 [==============================] - 1418s 6s/step - loss: 0.1162 - acc: 0.9610 - val_loss: 0.3963 - val_acc: 0.8473
Epoch 2/3
220/220 [==============================] - 1405s 6s/step - loss: 0.0551 - acc: 0.9800 - val_loss: 0.4723 - val_acc: 0.8410
Epoch 3/3
220/220 [==============================] - 1408s 6s/step - loss: 0.0411 - acc: 0.9886 - val_loss: 0.5046 - val_acc: 0.8812


In [52]:
model.save_weights('audio_cnn.h5')

In [53]:
test_audio,sf = lr.load('/Users/harsh/Downloads/Audio_1/sample_debate.mp3')

In [54]:
bins = 1024
seg = 43*5
fft = np.zeros((seg,bins//2),dtype='complex')
lfft = len(test_audio)//bins 
l = 0
for k in range(lfft//seg):
    while(l<seg*(k+1)):
        fft[l%seg] = np.abs(np.fft.fftshift(np.fft.fft(test_audio[l*bins:(l+1)*bins]))[bins//2:])*2/bins
        l+=1
    fig = plt.figure(k,frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(np.abs(fft.T),aspect='auto',clim=[0,5e-3],extent=[1,seg,256,1])
    ax.set_yscale('log')
    fig.savefig('/Users/harsh/Downloads/Audio_1/sample_debate_png/'+str(k)+'.png')
    print(float(k+1)/(lfft//seg)*100)
    plt.close()

0.0
0.5747126436781609
1.1494252873563218
1.7241379310344827
2.2988505747126435
2.8735632183908044
3.4482758620689653
4.022988505747127
4.597701149425287
5.172413793103448
5.747126436781609
6.321839080459771
6.896551724137931
7.471264367816093
8.045977011494253
8.620689655172415
9.195402298850574
9.770114942528735
10.344827586206897
10.919540229885058
11.494252873563218
12.068965517241379
12.643678160919542
13.218390804597702
13.793103448275861
14.367816091954023
14.942528735632186
15.517241379310345
16.091954022988507
16.666666666666664
17.24137931034483
17.81609195402299
18.39080459770115
18.96551724137931
19.54022988505747
20.114942528735632
20.689655172413794
21.26436781609195
21.839080459770116
22.413793103448278
22.988505747126435
23.563218390804597
24.137931034482758
24.71264367816092
25.287356321839084
25.862068965517242
26.436781609195403
27.011494252873565
27.586206896551722
28.160919540229884
28.735632183908045
29.310344827586203
29.88505747126437
30.45977011494253
31.034482

In [55]:
train_set.class_indices

{'Female': 0, 'Male': 1}

In [56]:
pred_map = {0:'Female',1:'Male'}

In [59]:
ftxt = open('/Users/harsh/Downloads/Audio_1/sample_debate_png/prediction.txt','w+')
for i in range(lfft//seg):
    pred_img = cv2.imread('/Users/harsh/Downloads/Audio_1/sample_debate_png/'+str(i)+'.png')
    pred_img = cv2.resize(pred_img,(512,512))
    pred_img = pred_img/255
    pred_img = np.expand_dims(pred_img,axis=0)
    pred = model.predict(pred_img)
    pred = 1 if(pred>0.5) else 0
    ftxt.write(str(i)+' : '+pred_map.get(pred)+'\n')  
#pred
ftxt.close()
    